<a href="https://colab.research.google.com/github/TausifShahanshah/Movie-Recommendation-using-PySpark-Big-Data/blob/main/Model_1_movie_recommender_using_spark_ori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install findspark

In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285397 sha256=fa18e90f9c95dd4d6eab2ad1c2d75bf931697fd970cb48496b3b9f63fb350309
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [ ]:
spark = SparkSession.builder.appName("recommendation").getOrCreate()


In [ ]:
data = spark.read.csv('/content/drive/MyDrive/PySpark_BigData_Movie_Recommendations/DataSet/ml-25m/ml-25m/ratings.csv',inferSchema=True,header=True)

In [ ]:
data.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
|     1|   1088|   4.0|1147868495|
|     1|   1175|   3.5|1147868826|
|     1|   1217|   3.5|1147878326|
|     1|   1237|   5.0|1147868839|
|     1|   1250|   4.0|1147868414|
|     1|   1260|   3.5|1147877857|
|     1|   1653|   4.0|1147868097|
|     1|   2011|   2.5|1147868079|
|     1|   2012|   2.5|1147868068|
|     1|   2068|   2.5|1147869044|
|     1|   2161|   3.5|1147868609|
|     1|   2351|   4.5|1147877957|
|     1|   2573|   4.0|1147878923|
|     1|   2632|   5.0|1147878248|
|     1|   2692|   5.0|1147869100|
+------+-------+------+----------+
only showing top 20 rows



In [ ]:

data.head()

Row(userId=1, movieId=296, rating=5.0, timestamp=1147880044)

In [ ]:

data.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [ ]:

data.describe().show()

+-------+-----------------+------------------+------------------+--------------------+
|summary|           userId|           movieId|            rating|           timestamp|
+-------+-----------------+------------------+------------------+--------------------+
|  count|         25000095|          25000095|          25000095|            25000095|
|   mean|81189.28115381162|21387.981943268616| 3.533854451353085|1.2156014431215513E9|
| stddev|46791.71589745776| 39198.86210105973|1.0607439611423535| 2.268758080595386E8|
|    min|                1|                 1|               0.5|           789652009|
|    max|           162541|            209171|               5.0|          1574327703|
+-------+-----------------+------------------+------------------+--------------------+



In [ ]:
# Smaller dataset so we will use 0.8 / 0.2
(train_data, test_data) = data.randomSplit([0.8, 0.2], seed=42)

In [ ]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(train_data)

In [ ]:

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test_data)

In [ ]:

predictions.show()

+------+-------+------+----------+----------+
|userId|movieId|rating| timestamp|prediction|
+------+-------+------+----------+----------+
|     1|    307|   5.0|1147868828| 4.0434666|
|     1|   1175|   3.5|1147868826|  4.469938|
|     1|   1237|   5.0|1147868839| 3.9969754|
|     1|   2012|   2.5|1147868068| 2.7791977|
|     1|   2692|   5.0|1147869100| 4.4192047|
|     1|   3949|   5.0|1147868678| 3.8163218|
|     1|   4973|   4.5|1147869080|  4.101731|
|     1|   5912|   3.0|1147878698|  4.912349|
|     1|   7318|   2.0|1147879850| 1.7260106|
|     1|   7323|   3.5|1147869119|  4.031815|
|     1|   7327|   3.5|1147868855|  3.641981|
|     1|   7365|   4.0|1147869033|  3.331927|
|     1|   7937|   3.0|1147878055|  3.346899|
|     1|   8014|   3.5|1147869155| 4.1159987|
|     1|   8786|   4.0|1147877853| 3.9367113|
|     1|  32591|   5.0|1147879538| 4.5896635|
| 32906|   1188|   5.0| 965797754|  4.416707|
| 32906|   1203|   5.0| 965802215|  4.896045|
| 32906|   1204|   5.0| 965798677|

In [ ]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"RMSE:{rmse}")
# print("Root-mean-square error = " + str(rmse))

RMSE:nan


RMSE describe our error in terms of star rating column

In [ ]:

single_user = test_data.filter(test_data['userId']==12).select(['movieId','userId'])

In [ ]:

# User had 10 ratings in the test data set
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      1|    12|
|     50|    12|
|     88|    12|
|    101|    12|
|    140|    12|
|    163|    12|
|    203|    12|
|    209|    12|
|    257|    12|
|    319|    12|
|    351|    12|
|    377|    12|
|    435|    12|
|    441|    12|
|    442|    12|
|    471|    12|
|    489|    12|
|    497|    12|
|    508|    12|
|    524|    12|
+-------+------+
only showing top 20 rows



In [ ]:

reccomendations = model.transform(single_user)

In [ ]:

reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|    926|    12| 4.2654324|
|    745|    12|  4.255452|
|   4973|    12| 4.2505665|
|   1361|    12|  4.192606|
|   1212|    12| 4.1667967|
|   1136|    12| 4.1640124|
|    858|    12|  4.104315|
|   1219|    12| 4.0647836|
|     50|    12| 4.0581856|
|   1225|    12| 4.0229697|
|    527|    12|  4.019446|
|   3996|    12| 4.0162525|
|   1175|    12| 3.9973946|
|   1269|    12| 3.9454193|
|  27790|    12| 3.9268074|
|   1258|    12| 3.9266632|
|   2788|    12|  3.921261|
|  34148|    12| 3.8974822|
|   6104|    12| 3.8962827|
|   8951|    12| 3.8767612|
+-------+------+----------+
only showing top 20 rows

